In [1]:
import numpy as np
import joblib
from skimage.feature import hog
from skimage import color
from skimage.transform import pyramid_gaussian
from imutils.object_detection import non_max_suppression
from scipy.special import expit
import cv2

In [2]:
model = joblib.load('svm_model3.npy')

orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

scale = 0
detections = []

(winW, winH) = (64,128)  # size of the sliding window
windowSize = (winW,winH)
downscale = 1.25 

In [3]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])

In [4]:
img = cv2.imread('C:\\Users\\admin\\JupyterNotebook_works\\mtech-programing-labs\\S2\\Computer-vision\\Project\\detection\\inputs\\1\\313.png')
img = cv2.resize(img,(300,200))

In [5]:
for resized in pyramid_gaussian(img, downscale=downscale): 
    for (x,y,window_) in sliding_window(resized, stepSize=10, windowSize=(winW,winH)):
        if window_.shape[0] != winH or window_.shape[1] !=winW or window_.shape[2]!=3:
            continue
        window = color.rgb2gray(window_)
        #window = cv2.cvtColor(window_, cv2.COLOR_BGR2GRAY) #.rgb2gray(window_[:,:,:3])
        fds = hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2') 
        fds = fds.reshape(1, -1)
        pred = model.predict(fds)
        
        if pred == 1:
            if expit(model.decision_function(fds))>=0.90: 
                print("Detection:: Location -> ({}, {})".format(x, y))
                print("Scale ->  {} | Confidence Score {} | Prob: {}\n".format(scale ,model.decision_function(fds),
                                                                              expit(model.decision_function(fds))))
                detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), model.decision_function(fds),
                                   int(windowSize[0]*(downscale**scale)),
                                      int(windowSize[1]*(downscale**scale))))
                
    scale+=1

Detection:: Location -> (60, 40)
Scale ->  0 | Confidence Score [2.33552453] | Prob: [0.91177674]

Detection:: Location -> (0, 50)
Scale ->  0 | Confidence Score [2.24796171] | Prob: [0.90447457]

Detection:: Location -> (120, 50)
Scale ->  0 | Confidence Score [3.24964143] | Prob: [0.96266023]

Detection:: Location -> (60, 20)
Scale ->  1 | Confidence Score [2.4495009] | Prob: [0.92052494]

Detection:: Location -> (40, 30)
Scale ->  1 | Confidence Score [3.03603191] | Prob: [0.95417564]

Detection:: Location -> (50, 30)
Scale ->  1 | Confidence Score [2.71323492] | Prob: [0.9378031]



In [6]:
rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections])
sc = [score[0] for (x, y, score, w, h) in detections]
print("detection confidence score: ", sc)
sc = np.array(sc)
nms_rects = non_max_suppression(rects, probs = sc, overlapThresh=0.3)

detection confidence score:  [2.3355245346000206, 2.2479617111265533, 3.2496414267593003, 2.449500895704287, 3.0360319122883475, 2.713234919730321]


In [26]:
rects

array([[ 60,  40, 124, 168],
       [  0,  50,  64, 178],
       [120,  50, 184, 178],
       [ 75,  25, 155, 185],
       [ 50,  37, 130, 197],
       [ 62,  37, 142, 197]])

In [7]:
nms_sc = []
for i in range(rects.shape[0]):
    for j in range(nms_rects.shape[0]):
        if np.array_equal(rects[i], nms_rects[j]):
            nms_sc.append(sc[i])

In [28]:
nms_rects

array([[120,  50, 184, 178],
       [ 50,  37, 130, 197],
       [  0,  50,  64, 178]])

In [8]:
for i, (xA, yA, xB, yB) in enumerate(nms_rects):
    print(i)
    cv2.rectangle(img, (xA, yA), (xB, yB), (0,255,0), 2)
    cv2.putText(img,'Human',(xA-2,yA-2),1,0.75,(255,255,0),1)

0
1
2


In [9]:
cv2.imshow('Detection', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
model = joblib.load('svm_model3.npy')

orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

(winW, winH) = (64,128)  # size of the sliding window
windowSize = (winW,winH)
downscale = 1.25

In [11]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])

In [12]:
def detect(img):
    scale = 0
    detections = []
    for resized in pyramid_gaussian(img, downscale=downscale): 
        for (x,y,window_) in sliding_window(resized, stepSize=10, windowSize=(winW,winH)):
            if window_.shape[0] != winH or window_.shape[1] !=winW or window_.shape[2]!=3:
                continue
            window = color.rgb2gray(window_)
            #window = cv2.cvtColor(window_, cv2.COLOR_BGR2GRAY) #.rgb2gray(window_[:,:,:3])
            fds = hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2') 
            fds = fds.reshape(1, -1)
            pred = model.predict(fds)

            if pred == 1:
                if expit(model.decision_function(fds))>=0.90:
                    #print("Detection:: Location -> ({}, {})".format(x, y))
                    #print("Scale ->  {} | Confidence Score {} \n".format(scale,model.decision_function(fds)))
                    detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), expit(model.decision_function(fds)),
                                       int(windowSize[0]*(downscale**scale)), 
                                          int(windowSize[1]*(downscale**scale))))
        scale+=1
    return detections

In [13]:
from datetime import datetime

In [14]:
# importing libraries
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
#cap = cv2.VideoCapture('C:\\Users\\admin\\JupyterNotebook_works\\mtech-programing-labs\\S2\\Computer-vision\\Project\\detection\\videoplayback.mp4')

cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    time_bf = datetime.now()
    ret, frame = cap.read()
    if ret == True:
        frame_res = cv2.resize(frame,(300,200))
        detections = detect(frame_res)
        rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections]) # do nms on the detected bounding boxes
        sc = [score[0] for (x, y, score, w, h) in detections]
        sc = np.array(sc)
        nms_rects = non_max_suppression(rects, probs = sc, overlapThresh=0.3)
        nms_sc = []
        for i in range(rects.shape[0]):
            for j in range(nms_rects.shape[0]):
                if np.array_equal(rects[i], nms_rects[j]):
                    nms_sc.append(sc[i])
        nms_sc = np.array(nms_sc)
        
        for i, (xA, yA, xB, yB) in enumerate(nms_rects):
            cv2.rectangle(frame_res, (xA, yA), (xB, yB), (0,255,0), 2)
            cv2.putText(frame_res,'Human',(xA-2,yA-2),1,1,(0,255,0),1)
            print(nms_sc[i])
        time_af = datetime.now()
        time_dif = time_af - time_bf
        print('Detection Time: {} ms'.format(round(time_dif.total_seconds()*1000, 3)))
        cv2.imshow('frame', frame_res)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    else:
        break
        
cap.release()
cv2.destroyAllWindows()

0.9211618339256893
0.946712380908483
0.9791766889049042
Detection Time: 2522.551 ms
0.91751865033145
Detection Time: 908.897 ms
0.9725327784883417
0.9251126988002584
Detection Time: 864.837 ms
0.9462559489016246
0.9501918715654489
Detection Time: 813.549 ms
0.9471732991642706
0.9235643165463869
Detection Time: 888.924 ms
0.962601272971059
0.9305414715938581
Detection Time: 905.227 ms
0.9574039453957844
Detection Time: 863.569 ms
0.992790098753327
0.9711031340773004
Detection Time: 811.335 ms
0.9339350474399053
0.9308348310142548
0.9485706022473269
Detection Time: 818.519 ms
0.9578387593408446
0.9585609166279919
Detection Time: 873.708 ms
0.9660272205339583
0.9803193559114712
Detection Time: 823.991 ms
0.979749993639893
0.9744955491852334
Detection Time: 802.251 ms
0.9792468493365035
Detection Time: 951.317 ms
0.9869463650251634
Detection Time: 815.036 ms
0.9514726738594436
Detection Time: 823.248 ms
Detection Time: 876.739 ms
0.9753437948772207
0.9638175478365552
0.9496927792505173
Det